In [ ]:
%load_ext autoreload
%autoreload 2
import os
import time
from scipy.spatial.transform import Rotation as R
import mujoco
import mujoco.viewer
import mujoco.renderer
import mujoco.glfw
import numpy as np
from sensors import Sensors
from point_cloud import PointCloud
import mediapy as media

%matplotlib widget
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
models_paths = os.listdir('ycb')

models = [models_paths[i] for i in range(3)]

In [ ]:
models = models_paths
models

In [ ]:
def new_xml(model, quat):
    xml =  f"""
    <mujoco>
        <asset>
            <mesh name="{model}" file="ycb/{model}" scale="1 1 1"/>
        </asset>
        <worldbody>
            <light name="top" pos="0 0 1"/>
            <geom pos="0 0 0" type="mesh" contype="0" conaffinity="0" group="1" density="0" mesh="{model}"
            quat="{quat[0]} {quat[1]} {quat[2]} {quat[3]}"
            />
                        
            <camera name="camera1" pos="-1 0.1 0" xyaxes="0 -0.5 0 -0.01 0 2" mode="fixed" fovy="60" />
            <camera name="camera2" pos="1 0.1 0" xyaxes="0 0.5 0 0.01 0 2" mode="fixed" fovy="60" />
            <camera name="camera3" pos="1 0.3 0.5" xyaxes="0 0.5 0 0.01 0 2" mode="fixed" fovy="60" />
            <camera name="camera4" pos="1 0.1 -0.2" xyaxes="0 0.5 0 0.01 0 2" mode="fixed" fovy="60" />
              
        </worldbody>
    </mujoco>
    """

    return xml

In [ ]:
import random

def get_elements_with_probability(arr=[0,1,2,3]):
    # Define the probabilities
    probabilities = [0.7, 0.25, 0.05]

    # Determine the number of elements to select based on probabilities
    num_elements = random.choices([1, 2, 3], probabilities)[0]

    # Shuffle the array to randomize the selection
    random.shuffle(arr)

    # Return a set of the selected elements
    return arr[:num_elements]

In [ ]:
from scipy.spatial import distance

def erase_outliers(point_cloud, threshold=0.5):

    # Calculate the mean and standard deviation of the point cloud
    mean = np.mean(point_cloud, axis=0)
    std_dev = np.std(point_cloud, axis=0)

    # Calculate the Mahalanobis distance for each point from the mean
    mahalanobis_distances = distance.cdist(point_cloud, [mean], 'mahalanobis', VI=np.linalg.inv(np.diag(std_dev)))

    # Create a mask to identify outliers
    outlier_mask = np.squeeze(mahalanobis_distances > threshold)
    
    # Filter out the outliers from the point cloud
    filtered_point_cloud = point_cloud[~outlier_mask]

    return filtered_point_cloud

In [ ]:
m = mujoco.MjModel.from_xml_string(new_xml(models[0], [1, 0, 0, 0]))
m.cam(0).pos

m.cam_pos[0]

In [ ]:
# Random quaternion
quat = np.random.random(4)
quat = quat / np.linalg.norm(quat)

m = mujoco.MjModel.from_xml_string(new_xml("070-a_colored_wood_blocks.stl", quat))
d = mujoco.MjData(m)

# Launching the viewer
with mujoco.viewer.launch_passive(m, d) as viewer:


  # Close the viewer automatically after 30 wall-seconds.
  start = time.time()


  while viewer.is_running():

    step_start = time.time()


    # mj_step can be replaced with code that also evaluates
    # a policy and applies a control signal before stepping the physics.
    mujoco.mj_step(m, d)
    
    # Pick up changes to the physics state, apply perturbations, update options from GUI.
    viewer.sync()


    # Rudimentary time keeping, will drift relative to wall clock.
    time_until_next_step = abs(m.opt.timestep - (time.time() - step_start))

    current_time = time.time() - start
    while d.time < current_time:
      mujoco.mj_step(m, d)

    if time_until_next_step > 0:
      time.sleep(time_until_next_step)

In [ ]:
# Creating instance of classes for data collection
sensors = Sensors()
point_c = PointCloud((480, 640), 60, downsample=3)

for model in models:
    name = model.split(".")[0]
    for n in range(2):
        # Generating random quaternion
        quat = np.random.random(4)
        quat = quat / np.linalg.norm(quat)

        # Generating new xml
        xml = new_xml(model, quat)

        # Create the model and data objects.
        m = mujoco.MjModel.from_xml_string(xml)
        d = mujoco.MjData(m)

        # Make renderer, render and show the pixels
        renderer = mujoco.Renderer(m,480, 640)
        depth = mujoco.Renderer(m,480, 640); depth.enable_depth_rendering()
        sgmnt = mujoco.Renderer(m,480, 640); sgmnt.enable_segmentation_rendering()
        
        # Doing initial loading
        mujoco.mj_step(m, d)

        #TODO: Get point cloud from n out of m random cameras
        n_cameras = m.cam_user.shape[0]

        # This can be obtained from the model at the beginning of the simulation
        # Or thery can be retrieved at any time during the simulation
        # Get the position and rotation of the cameras
        cameras = get_elements_with_probability()
        poses = []
        rotations = []
        rot_matrices = []
        for i in cameras:
            # Get the position of the camera
            pos = m.cam_pos[i] + m.body(m.cam_bodyid[i]).pos
            poses.append(pos)

            # Get the rotation of the camera
            angle = R.from_quat(m.cam(i).quat).as_euler('xyz', degrees=True)
            rot_vect = R.from_quat(m.cam(i).quat).as_matrix()
            rotations.append(angle)
            rot_matrices.append(rot_vect)

        # Renderer with no additional features
        # frames = [*sensors.get_rgb_image_matrices(m, d, renderer, cameras).values()]
        # for frame in frames:
        #     media.show_image(frame)

        # Renderer with depth enabled
        depth_frames = [*sensors.get_depth_image_matrices(m, d, depth, cameras).values()]

        # Renderer with segmentation enabled
        sgmnt_frames = [*sensors.get_segment_image_matrices(m, d, sgmnt, cameras).values()]
        
        # Get the point cloud from the segmented map
        points = [*point_c.get_segmented_map(depth_frames, sgmnt_frames, poses, rot_matrices, m).values()]

        points = np.array(points[0])

        # Save the point cloud
        # np.save(f'dataset_pc/{name}_{"%06d" % (n+1,)}.npy', points[0])
        
        # Create a 3D figure
        mat = np.array(points)

        # mat = erase_outliers(mat, threshold=1)

        # Create a 3D figure
        fig = plt.figure()
        ax = fig.add_subplot(projection='3d')

        # Define the x, y, z coordinates of the point cloud
        x = mat[:, 0]
        y = mat[:, 1]
        z = mat[:, 2]

        # Plot the point cloud data
        ax.scatter(x, y, z, s=1)

        # Set the axis labels
        ax.set_xlabel('X Label')
        ax.set_ylabel('Y Label')
        ax.set_zlabel('Z Label')

        ax.set_xlim(-0.3, 0.3)
        ax.set_ylim(-0.3, 0.3)
        ax.set_zlim(-0.3, 0.3)

        # Show the plot
        plt.show()
            

        # print(model)
    

In [ ]:
arr = np.array([[1],[0],[-1]])

mult = np.array([])

In [ ]:
from vectorized_point_cloud import VectorizedPC

pc = VectorizedPC((10, 10), 60)

frame = np.ones((10, 10))

rotation = np.array([
    [0.5, 0, 0],
    [0, 0.5, 0],
    [0, 0, 0.5]
])

translation = np.array([0.1, 0.1, 2])

pc.get_points(frame, rotation, translation)

In [ ]:
quat = np.random.random(4)
quat = quat / np.linalg.norm(quat)

# Generating new xml
xml = new_xml("011_banana.stl", quat)

# Create the model and data objects.
m = mujoco.MjModel.from_xml_string(xml)
d = mujoco.MjData(m)

# Make renderer, render and show the pixels
renderer = mujoco.Renderer(m,480, 640)
depth = mujoco.Renderer(m,480, 640); depth.enable_depth_rendering()
sgmnt = mujoco.Renderer(m,480, 640); sgmnt.enable_segmentation_rendering()

# Doing initial loading
mujoco.mj_step(m, d)

sensors = Sensors()

frame = [*sensors.get_rgb_image_matrices(m, d, renderer, [0,1]).values()]

depth = [*sensors.get_depth_image_matrices(m, d, depth, [0,1]).values()]
rot_mat = R.from_quat(m.cam(0).quat).as_matrix()
pos = m.cam_pos[0]

In [ ]:
depth.shape

In [ ]:
pc = VectorizedPC((480, 640), 60)


mat = pc.get_points(depth, rot_mat, pos)
mat

In [ ]:
# Create a 3D figure
fig = plt.figure()
ax = fig.add_subplot(projection='3d')

# Define the x, y, z coordinates of the point cloud
x = mat[:, 0]
y = mat[:, 1]
z = mat[:, 2]

# Plot the point cloud data
ax.scatter(x, y, z, s=1)

# Set the axis labels
ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')

ax.scatter(0, 0, 0, s=10)

# ax.set_xlim(-0.3, 0.3)
# ax.set_ylim(-0.3, 0.3)
# ax.set_zlim(-0.3, 0.3)

# Show the plot
plt.show()